In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
from datetime import time

# === 1. DB CONFIGURATION ===
DB_HOST = '192.168.18.206'
DB_PORT = 3306
DB_USER = 'root'
DB_PASSWORD = 'mydb1234@XTI#2025'
DB_NAME = 'hrbo'

# Encode password for URL
encoded_password = urllib.parse.quote_plus(DB_PASSWORD)

# Create SQLAlchemy engine
engine = create_engine(
    f"mysql+mysqlconnector://{DB_USER}:{encoded_password}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# === 2. READ EXCEL FILE ===
EXCEL_PATH = 'employees.xlsx'  # Make sure this file exists in the same directory
df_excel = pd.read_excel(EXCEL_PATH)

# Normalize column names
df_excel.columns = [col.strip().lower().replace(" ", "_") for col in df_excel.columns]  # expecting 'employee_id' and 'name'

# === 3. FETCH EXISTING EMPLOYEE IDs FROM regular_schedule ===
existing_ids = pd.read_sql("SELECT DISTINCT employee_id FROM hrbo.regular_schedule", engine)
existing_ids_set = set(existing_ids['employee_id'])

print(len(existing_ids_set))
# # === 4. FILTER EMPLOYEES NOT IN SCHEDULE ===
new_employees = df_excel[~df_excel['employee_id'].isin(existing_ids_set)]

# === 5. CREATE SCHEDULE FOR MONDAY TO SUNDAY ===
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
in_time = time(21, 15)
out_time = time(6, 0)

rows = []
for _, row in new_employees.iterrows():
    for day in days:
        rows.append({
            'employee_id': row['employee_id'],
            'employee_name': row['name'],
            'day_of_week': day,
            'scheduled_in_time': in_time,
            'scheduled_out_time': out_time
        })

df_to_insert = pd.DataFrame(rows)

# === 6. INSERT INTO hrbo.regular_schedule ===
df_to_insert.to_sql(
    name='regular_schedule',
    con=engine,
    schema='hrbo',
    if_exists='append',
    index=False,
    method='multi'
)

print(f"✅ Inserted {len(df_to_insert)} rows into hrbo.regular_schedule.")


120


In [24]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
from datetime import time


# === READ EXCEL FILE ===
EXCEL_PATH = 'regular_schedule_clean.xlsx'  # Make sure this file exists in the same directory
df_excel = pd.read_excel(EXCEL_PATH)
df_excel.columns = df_excel.columns.str.lower()
df_excel['scheduled_in_time'] = pd.to_datetime(df_excel['scheduled_in_time']).dt.time
df_excel['scheduled_in_time'] = df_excel['scheduled_in_time'].apply(
    lambda x: (pd.Timestamp.combine(pd.Timestamp.today(), x) + pd.Timedelta(minutes=15)).time() if x != time(0, 0) else x
)

C:\Users\Desktop\AppData\Local\Temp\ipykernel_1404\1310458808.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_excel['scheduled_in_time'] = pd.to_datetime(df_excel['scheduled_in_time']).dt.time


In [25]:
df_excel

,employee_id,employee_name,day_of_week,scheduled_in_time,scheduled_out_time
0,39,Fahad Bin Zahid,Monday,20:15:00,05:00:00
1,39,Fahad Bin Zahid,Tuesday,20:15:00,05:00:00
2,39,Fahad Bin Zahid,Wednesday,20:15:00,05:00:00
3,39,Fahad Bin Zahid,Thursday,20:15:00,05:00:00
4,39,Fahad Bin Zahid,Friday,20:15:00,05:00:00
...,...,...,...,...,...
716,997,salman,Wednesday,20:15:00,05:00:00
717,997,salman,Thursday,20:15:00,05:00:00
718,997,salman,Friday,20:15:00,05:00:00
719,997,salman,Saturday,20:15:00,05:00:00


In [26]:
try:
    DB_HOST = '192.168.18.206'
    DB_PORT = 3306
    DB_USER = 'root'
    DB_PASSWORD = 'mydb1234@XTI#2025'
    DB_NAME = 'hrbo'

    encoded_password = urllib.parse.quote_plus(DB_PASSWORD)

    engine = create_engine(
        f"mysql+mysqlconnector://{DB_USER}:{encoded_password}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    )

    df_excel.to_sql(
        name='regular_schedule',
        con=engine,
        schema='hrbo',
        if_exists='append',
        index=False,
        method='multi'
    )
    print(f"✅ Inserted {len(df_excel)} rows into hrbo.regular_schedule.")
except Exception as e:
    print(f"❌ Error occurred: {e}")

✅ Inserted 721 rows into hrbo.regular_schedule.


In [39]:
EXCEL_PATH = 'regular_schedule_clean.xlsx'  # Make sure this file exists in the same directory
df_excel = pd.read_excel(EXCEL_PATH)
df_excel.columns = df_excel.columns.str.lower()
df_excel['scheduled_in_time'] = pd.to_datetime(df_excel['scheduled_in_time']).dt.time
df_excel['scheduled_in_time'] = df_excel['scheduled_in_time'].apply(
    lambda x: (pd.Timestamp.combine(pd.Timestamp.today(), x) + pd.Timedelta(minutes=15)).time() if x != time(0, 0) else x
)

C:\Users\Desktop\AppData\Local\Temp\ipykernel_1404\3704689960.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_excel['scheduled_in_time'] = pd.to_datetime(df_excel['scheduled_in_time']).dt.time


In [40]:
from datetime import time

# Get employee names where none of the 7-day entries is 00:00:00 (no holiday)
employees_no_holiday = df_excel.groupby('employee_name').filter(
    lambda group: all(t != time(0, 0) for t in group['scheduled_in_time'])
)

# Show only relevant columns
# employees_no_holiday[['employee_name', "day_of_week",'scheduled_in_time']]
employees_no_holiday['employee_name'].nunique() 



7